<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1">Подготовка</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.0.1">Вывод:</a></span></li></ul></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2">Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1">LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.2">DecisionTreeClassifier</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.3">LGBMClassifier</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.4">Вывод:</a></span></li><li><span><a href="#Предсказание-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Предсказание-лучшей-модели-на-тестовой-выборке-2.5">Предсказание лучшей модели на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3">Выводы</a></span></li></ul></div>

# Проект

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

In [1]:
#Загрузим необходимые библиотеки
import numpy as np
import pandas as pd
import re
import time
import nltk
nltk.download('averaged_perceptron_tagger')
from lightgbm import LGBMClassifier
from nltk.stem import WordNetLemmatizer #Лемматизация
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords #Список стоп-слов
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve, classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
RANDOM_STATE = 12345


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#Загрузим предоставленный нам датасет
tox_df = pd.read_csv('/datasets/toxic_comments.csv')
tox_df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
tox_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
tox_df.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


In [5]:
# Количество токичных и нетоксичных комментариев
tox_df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
# Проверим на дубликаты
tox_df.duplicated().sum()

0

In [7]:
tox_df.index.duplicated().sum()

0

#### Вывод:
Дубликаты отсутствуют.
Данные не сбалансированы.

 <div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Данные загружены корреткно. Молодец, что проверяешь баланс классов)

In [8]:
# Лемматизируем данные
# Функция для преобразования POS тегов из nltk в формат WordNet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Лемматизируем текст с использованием POS тэгов
def lemmatize_text(text):
    text = text.lower()  # Привести к нижнему регистру
    tokens = text.split()  # Разбиваем на токены
    pos_tags = pos_tag(tokens)  # Получаем POS теги
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in pos_tags]
    cleared_text = re.sub(r"[^a-zA-Z]", ' ', ' '.join(lemmatized_tokens))
    return ' '.join(cleared_text.split())

tox_df['lemm_text'] = tox_df['text'].apply(lemmatize_text)
tox_df = tox_df.drop(['text'], axis=1)

In [9]:
# Разобьем данные на выборки 
target = tox_df['toxic']
features = tox_df.drop(['toxic'], axis=1)

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.2, 
                                                                              random_state=12345)

In [10]:
# Обработаем текстовые данные для задач машинного обучения
# Загрузка стоп-слов
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

# Создание TF-IDF-векторизатора
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

# Обучение и преобразование данных
features_train = count_tf_idf.fit_transform(features_train['lemm_text'].values)
features_test = count_tf_idf.transform(features_test['lemm_text'].values)
print(features_train.shape)
print(features_test.shape)
cv_counts = 2

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(127433, 143002)
(31859, 143002)


In [11]:
# Выполним кросс-валидацию модели машинного обучения и оценим её производительность по метрике F1
classificator = LogisticRegression()
train_f1 = cross_val_score(classificator, 
                      features_train, 
                      target_train, 
                      cv=cv_counts, 
                      scoring='f1').mean()
print('F1 на CV', train_f1)

F1 на CV 0.6758897106059467


## Обучение

In [12]:
# Создадим функцию для создания, обучения и оценки модели, а также сохранения результатов
# Инициализация DataFrame для хранения результатов
df_results = pd.DataFrame(columns=['name_model', 'best_score_', 'parameters', 'time_c'])

def create_model(classificator, param_grid, model_name, features_train, target_train):
    '''
    Параметры:
    classificator -- модель классификатора
    param_grid -- сетка гиперпараметров для оптимизации
    model_name -- название модели для вывода
    features_train -- обучающая выборка признаков
    targets_train -- обучающая выборка меток
    '''
    # Начало работы с индикатором выполнения
    start_time = time.time()
    
    # Настройка GridSearchCV для поиска лучших параметров
    grid_search = GridSearchCV(classificator, param_grid, scoring='f1', cv=3, return_train_score=True)
    
    # Обучение модели
    grid_search.fit(features_train, target_train)
    
    # Расчет времени выполнения
    elapsed_time = time.time() - start_time
    
    # Извлечение лучших параметров и лучшего результата
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    # Добавление результатов в DataFrame, если такая модель еще не добавлена
    if model_name not in list(df_results['name_model']):
        df_results.loc[len(df_results.index)] = [model_name, best_score, best_params, elapsed_time]
    
    # Вывод результатов
    display(df_results)


### LogisticRegression

In [13]:
logreg = LogisticRegression(fit_intercept=True, 
                                class_weight='balanced', 
                                random_state=RANDOM_STATE,
                                solver='liblinear'
                                )
logreg_param = {'C': [1, 10]}   
create_model(logreg, logreg_param, 'LogisticRegression', features_train, target_train)

,name_model,best_score_,parameters,time_c
0,LogisticRegression,0.75529,{'C': 10},135.288768


### DecisionTreeClassifier

In [14]:
dtc = DecisionTreeClassifier(class_weight='balanced', 
                                      random_state=RANDOM_STATE
                                      )
dtc_param = {'max_depth': [30]}
    
create_model(dtc, dtc_param, 'DecisionTreeClassifier', features_train, target_train)

,name_model,best_score_,parameters,time_c
0,LogisticRegression,0.755290,{'C': 10},135.288768
1,DecisionTreeClassifier,0.617569,{'max_depth': 30},103.038885


### LGBMClassifier

In [15]:
lgbm = LGBMClassifier(learning_rate=0.15, 
                      random_state=RANDOM_STATE,
                      n_jobs=-1
                      )
lgbm_param = {'max_depth': [10],
                   'n_estimators': [100]
                  }
    
create_model(lgbm, lgbm_param, 'LGBMClassifier', features_train, target_train)

,name_model,best_score_,parameters,time_c
0,LogisticRegression,0.755290,{'C': 10},135.288768
1,DecisionTreeClassifier,0.617569,{'max_depth': 30},103.038885
2,LGBMClassifier,0.709030,"{'max_depth': 10, 'n_estimators': 100}",489.437148


### Вывод:
Лучшей моделью по метрике F1 и времени обучения оказалась логическая регрессия 

### Предсказание лучшей модели на тестовой выборке

In [16]:
best_logreg = LogisticRegression(
    fit_intercept=True,
    class_weight='balanced',
    random_state=RANDOM_STATE,
    solver='liblinear',
    C=10  # Лучшее значение гиперпараметра C
)

# Обучаем модель на тренировочных данных
best_logreg.fit(features_train, target_train)

# Выполняем предсказание на тестовой выборке
predictions = best_logreg.predict(features_test)

# Рассчитываем метрику F1 на тестовой выборке
f1 = f1_score(target_test, predictions)
print("F1 метрика на тестовой выборке:", f1)

F1 метрика на тестовой выборке: 0.7651601830663617


## Выводы

В ходе работы мы выполнили следующие действия:
- Импортировали необходимые нам библиотеки
- Загрузили датасет с данными и изучили его
- Количество токсичных комментариев - **143106**, нетоксичных - **16186**
- Проверили данные на пропуски и дубликаты
- Лемматизировали данные
- Разбили данные на обучающую и тестовую выборки\

Лучшую модель искали среди **LogisticRegression**, **DecisionTreeClassifier** и **LGBMClassifier**\
Лучшие гиперпараметры искали с помощью кроссвалидации
 

In [17]:
df_results

,name_model,best_score_,parameters,time_c
0,LogisticRegression,0.755290,{'C': 10},135.288768
1,DecisionTreeClassifier,0.617569,{'max_depth': 30},103.038885
2,LGBMClassifier,0.709030,"{'max_depth': 10, 'n_estimators': 100}",489.437148


В итоге лучшей оказалась модель логической регрессии с параметром **'C': 10**, которая показала результат F1-метрики в 0.76 при кроссвалидации и относительно небольшим временем обучения.

На тестовой выборке F1 равен **0.77479**

Интернет-магазину рекомендуется использовать модель логической регрессии для разделения комментариев на токсичные и нормальные.